In [2]:
%matplotlib inline

import sys
import time

import gym
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# load environment
env = gym.envs.make("MountainCar-v0")

In [4]:
# learn about the environment
# observation (state) is (position, velocity)
print(env.action_space)
print(env.observation_space)
print(env.observation_space.high)
print(env.observation_space.low)
print(env.observation_space.bounded_above)
print(env.observation_space.bounded_below)

Discrete(3)
Box(2,)
[0.6  0.07]
[-1.2  -0.07]
[ True  True]
[ True  True]


In [6]:
space = gym.spaces.Discrete(3)
x = space.sample()
print(space.contains(x))
print(space.n == 3)

True
True


In [20]:
# test the interaction with the environment
env.reset()
for i in range(200):
    env.render()
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    print("\r({}, {}, {}, {})".format(observation, reward, done, info), end="")
    time.sleep(0.01)
    
    if done:
        env.reset()


([-0.58936362 -0.00381573], -1.0, True, {'TimeLimit.truncated': True})